stdin, stdout, stderr = client.exec_command()

print_std(stdout, stderr)

In [22]:
import paramiko, json

with open("../01-deploy-aws-infra/inventory.json", 'r') as file:
    _DATA = json.load(file)
_SSH_KEY = _DATA["KeyPairPath"]
_HOST = _DATA["Instances"][0]["InstanceIp"]
_PORT = 22
_USER = "ubuntu"

In [2]:
def print_std(stdout, stderr):
    stdout = stdout.readlines()
    stderr = stderr.readlines()
    print("\t*** STDOUT ***")
    [print(i) for i in stdout]
    print("\n\t*** STDERR ***")
    [print(i) for i in stderr]
    
def get_stdout(stdout):
    stdout = stdout.readline()
    return stdout

# Config Master + Slave

In [3]:
client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(hostname=_HOST, username=_USER, key_filename=_SSH_KEY)





In [4]:
stdin, stdout, stderr = client.exec_command("swapoff -a")
print_std(stdout, stderr)

	*** STDOUT ***

	*** STDERR ***


In [5]:
stdin, stdout, stderr = client.exec_command("hostname -i")
host_ip = get_stdout(stdout)
print(host_ip)


192.168.0.171



## Installation Docker

In [6]:
stdin, stdout, stderr = client.exec_command("wget -qO- https://get.docker.com/ | sh")
print_std(stdout, stderr)

	*** STDOUT ***
# Executing docker install script, commit: 4f282167c425347a931ccfd95cc91fab041d414f

Client: Docker Engine - Community

 Version:           20.10.22

 API version:       1.41

 Go version:        go1.18.9

 Git commit:        3a2c30b

 Built:             Thu Dec 15 22:28:04 2022

 OS/Arch:           linux/amd64

 Context:           default

 Experimental:      true



Server: Docker Engine - Community

 Engine:

  Version:          20.10.22

  API version:      1.41 (minimum version 1.12)

  Go version:       go1.18.9

  Git commit:       42c8b31

  Built:            Thu Dec 15 22:25:49 2022

  OS/Arch:          linux/amd64

  Experimental:     false

 containerd:

  Version:          1.6.14

  GitCommit:        9ba4b250366a5ddde94bb7c9d1def331423aa323

 runc:

  Version:          1.1.4

  GitCommit:        v1.1.4-0-g5fd4c4d

 docker-init:

  Version:          0.19.0

  GitCommit:        de40ad0






To run Docker as a non-privileged user, consider setting up the

Dock

## Installation Kubernetes

In [7]:
stdin, stdout, stderr = client.exec_command("modprobe br_netfilter")
print_std(stdout, stderr)

stdin, stdout, stderr = client.exec_command("""
cat <<EOF | sudo tee /etc/sysctl.d/k8s.conf
net.bridge.bridge-nf-call-ip6tables = 1
net.bridge.bridge-nf-call-iptables = 1 
EOF""")
print_std(stdout, stderr)


	*** STDOUT ***

	*** STDERR ***
	*** STDOUT ***
net.bridge.bridge-nf-call-ip6tables = 1

net.bridge.bridge-nf-call-iptables = 1 


	*** STDERR ***


In [8]:
stdin, stdout, stderr = client.exec_command("sudo sysctl --system")
print_std(stdout, stderr)

	*** STDOUT ***
* Applying /etc/sysctl.d/10-console-messages.conf ...

kernel.printk = 4 4 1 7

* Applying /etc/sysctl.d/10-ipv6-privacy.conf ...

net.ipv6.conf.all.use_tempaddr = 2

net.ipv6.conf.default.use_tempaddr = 2

* Applying /etc/sysctl.d/10-kernel-hardening.conf ...

kernel.kptr_restrict = 1

* Applying /etc/sysctl.d/10-magic-sysrq.conf ...

kernel.sysrq = 176

* Applying /etc/sysctl.d/10-network-security.conf ...

net.ipv4.conf.default.rp_filter = 2

net.ipv4.conf.all.rp_filter = 2

* Applying /etc/sysctl.d/10-ptrace.conf ...

kernel.yama.ptrace_scope = 1

* Applying /etc/sysctl.d/10-zeropage.conf ...

vm.mmap_min_addr = 65536

* Applying /usr/lib/sysctl.d/50-default.conf ...

kernel.core_uses_pid = 1

net.ipv4.conf.default.rp_filter = 2

net.ipv4.conf.default.accept_source_route = 0

net.ipv4.conf.default.promote_secondaries = 1

net.ipv4.ping_group_range = 0 2147483647

net.core.default_qdisc = fq_codel

fs.protected_hardlinks = 1

fs.protected_symlinks = 1

fs.protected_r

In [9]:
stdout, stdin, stderr = client.exec_command("curl -s https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -")
print_std(stdin, stderr)

stdout, stdin, stderr = client.exec_command("""
cat <<EOF | sudo tee /etc/apt/sources.list.d/kubernetes.list
deb https://apt.kubernetes.io/ kubernetes-xenial main
EOF""")
print_std(stdin, stderr)

	*** STDOUT ***
OK


	*** STDERR ***

	*** STDOUT ***
deb https://apt.kubernetes.io/ kubernetes-xenial main


	*** STDERR ***


In [10]:
stdout, stdin, stderr = client.exec_command("sudo apt-get update")
print_std(stdin, stderr)

stdout, stdin, stderr = client.exec_command("sudo apt-get install -y kubelet kubeadm kubectl")
print_std(stdin, stderr)

	*** STDOUT ***
Hit:1 https://download.docker.com/linux/ubuntu jammy InRelease

Hit:2 http://eu-west-3.ec2.archive.ubuntu.com/ubuntu jammy InRelease

Hit:3 http://eu-west-3.ec2.archive.ubuntu.com/ubuntu jammy-updates InRelease

Hit:4 http://eu-west-3.ec2.archive.ubuntu.com/ubuntu jammy-backports InRelease

Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease

Get:6 https://packages.cloud.google.com/apt kubernetes-xenial InRelease [9383 B]

Get:7 https://packages.cloud.google.com/apt kubernetes-xenial/main amd64 Packages [62.2 kB]

Fetched 71.6 kB in 1s (119 kB/s)

Reading package lists...


	*** STDERR ***
W: https://apt.kubernetes.io/dists/kubernetes-xenial/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.

	*** STDOUT ***
Reading package lists...

Building dependency tree...

Reading state information...

The following additional packages will be installed:

  conntrack cri-tools ebtables 

In [11]:
stdin, stdout, stderr = client.exec_command("sudo sed -i 's|Environment=\"KUBELET_CONFIG_ARGS=--config=/var/lib/kubelet/config.yaml\"|Environment=\"KUBELET_CONFIG_ARGS=--config=/var/lib/kubelet/config.yaml --cgroup-driver=cgroupfs\"|' /etc/systemd/system/kubelet.service.d/10-kubeadm.conf")
print_std(stdout, stderr)

	*** STDOUT ***

	*** STDERR ***


In [12]:
stdin, stdout, stderr = client.exec_command("sudo systemctl daemon-reload")
print_std(stdout, stderr)

	*** STDOUT ***

	*** STDERR ***


In [13]:
client.close()

# Configuration Master

In [23]:
client.connect(hostname=_HOST, username=_USER, key_filename=_SSH_KEY)


In [18]:
stdin, stdout, stderr = client.exec_command("sudo rm /etc/containerd/config.toml && sudo systemctl restart containerd")
print_std(stdout, stderr)

	*** STDOUT ***

	*** STDERR ***


In [28]:
stdin, stdout, stderr = client.exec_command("sudo kubeadm init --pod-network-cidr=10.244.0.0/16 --ignore-preflight-errors=all --v=5")
print_std(stdout, stderr)

	*** STDOUT ***
[init] Using Kubernetes version: v1.26.0

[preflight] Running pre-flight checks

[preflight] Pulling images required for setting up a Kubernetes cluster

[preflight] This might take a minute or two, depending on the speed of your internet connection

[preflight] You can also perform this action in beforehand using 'kubeadm config images pull'

[certs] Using certificateDir folder "/etc/kubernetes/pki"

[certs] Generating "ca" certificate and key

[certs] Generating "apiserver" certificate and key

[certs] apiserver serving cert is signed for DNS names [ip-192-168-0-171 kubernetes kubernetes.default kubernetes.default.svc kubernetes.default.svc.cluster.local] and IPs [10.96.0.1 192.168.0.171]

[certs] Generating "apiserver-kubelet-client" certificate and key

[certs] Generating "front-proxy-ca" certificate and key

[certs] Generating "front-proxy-client" certificate and key

[certs] Generating "etcd/ca" certificate and key

[certs] Generating "etcd/server" certificate and

In [30]:
stdin, stdout, stderr = client.exec_command("sudo kubeadm token create --print-join-command")
master_join_command = get_stdout(stdout)
print(master_join_command)

kubeadm join 192.168.0.171:6443 --token fvuoh0.10c5uzprt522cut1 --discovery-token-ca-cert-hash sha256:9dc76c71cc062c578115746aba34d1b8587278eb7876ad70958eb371d139e268 

	*** STDOUT ***

	*** STDERR ***


In [32]:
stdin, stdout, stderr = client.exec_command("sudo mkdir -p $HOME/.kube && sudo cp -i /etc/kubernetes/admin.conf $HOME/.kube/config && sudo chown $(id -u):$(id -g) $HOME/.kube/config")
print_std(stdout, stderr)

	*** STDOUT ***

	*** STDERR ***


In [34]:
stdin, stdout, stderr = client.exec_command("kubectl get nodes && kubectl get pods --all-namespaces")
print_std(stdout, stderr)

	*** STDOUT ***
NAME               STATUS     ROLES           AGE     VERSION

ip-192-168-0-171   NotReady   control-plane   5m41s   v1.26.0

NAMESPACE     NAME                                       READY   STATUS    RESTARTS   AGE

kube-system   coredns-787d4945fb-9hsbm                   0/1     Pending   0          5m24s

kube-system   coredns-787d4945fb-lplvb                   0/1     Pending   0          5m24s

kube-system   etcd-ip-192-168-0-171                      1/1     Running   0          5m37s

kube-system   kube-apiserver-ip-192-168-0-171            1/1     Running   0          5m37s

kube-system   kube-controller-manager-ip-192-168-0-171   1/1     Running   0          5m37s

kube-system   kube-proxy-2z9sn                           1/1     Running   0          5m24s

kube-system   kube-scheduler-ip-192-168-0-171            1/1     Running   0          5m37s


	*** STDERR ***


In [36]:
stdin, stdout, stderr = client.exec_command("kubectl apply -f https://raw.githubusercontent.com/coreos/flannel/master/Documentation/kube-flannel.yml")
print_std(stdout, stderr)

	*** STDOUT ***
namespace/kube-flannel created

clusterrole.rbac.authorization.k8s.io/flannel created

clusterrolebinding.rbac.authorization.k8s.io/flannel created

serviceaccount/flannel created

configmap/kube-flannel-cfg created

daemonset.apps/kube-flannel-ds created


	*** STDERR ***


In [37]:
stdin, stdout, stderr = client.exec_command("kubectl get nodes && kubectl get pods --all-namespaces")
print_std(stdout, stderr)

	*** STDOUT ***
NAME               STATUS     ROLES           AGE    VERSION

ip-192-168-0-171   NotReady   control-plane   7m2s   v1.26.0

NAMESPACE      NAME                                       READY   STATUS     RESTARTS   AGE

kube-flannel   kube-flannel-ds-jj7sd                      0/1     Init:0/2   0          4s

kube-system    coredns-787d4945fb-9hsbm                   0/1     Pending    0          6m46s

kube-system    coredns-787d4945fb-lplvb                   0/1     Pending    0          6m46s

kube-system    etcd-ip-192-168-0-171                      1/1     Running    0          6m59s

kube-system    kube-apiserver-ip-192-168-0-171            1/1     Running    0          6m59s

kube-system    kube-controller-manager-ip-192-168-0-171   1/1     Running    0          6m59s

kube-system    kube-proxy-2z9sn                           1/1     Running    0          6m46s

kube-system    kube-scheduler-ip-192-168-0-171            1/1     Running    0          6m59s


	*** STDE